In [1]:
import os, sys, glob

print(os.getcwd())
os.chdir('..')
print(f'Changed dir to: {os.getcwd()}')
# !export PYTHONPATH="$(pwd)/src:$PYTHONPATH"

/Users/shayanriyaz/Documents/Projects/heart_rhythm_analysis/notebooks
Changed dir to: /Users/shayanriyaz/Documents/Projects/heart_rhythm_analysis


In [2]:
import numpy as np
import pandas as pd
from IPython.display import display
from random import choice
import scipy
project_root = os.path.abspath(os.path.join(os.getcwd()))

src_dir = os.path.join(project_root, "src")
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

from src.lib.create_training_pipelines.capnobase_etl import CapnoBaseETL
from src.lib.create_training_pipelines.mimic_etl import MimicETL
from src.lib.utils.timeseries_viewer import make_window_figure, create_time_series_viewer

import dash
app = dash.Dash(__name__)

import h5py, inspect
open_files = [v for v in globals().values() 
              if isinstance(v, h5py.File)]

for f in open_files:
    print(f"Closing {f.filename}")
    f.close()

bSetUpDB = True
WINDOW_LEN = 8
FOLDER_NAME = '_processed'

SCALE_TYPE="standard"
DECIMATE_SIGNAL=True
ZEROPHASE=True
LOW_PASS_CUTOFF=8.0
FIR_NUMTAPS=33

## Create Capno Dataset

### CapnoBase

In [4]:
# def main():
#     root_path = os.path.join('../data/raw/capnobase/data/mat')
#     out_path = os.path.join('../data/processed/length_full/capnobase_db')
#     out_filename = 'capnobase_db'
#     if not os.path.exists(out_path):
#         os.mkdir(out_path)

#     fs_in = 100.00
#     fs_out = 100.00

#     config = {
#     "input_dir"      : root_path,
#     "output_dir"     : out_path,
#     "window_size_sec": 30,
#     "fs_in"          : fs_in,
#     "fs_out"   : fs_out,
#     "lowpass_cutoff" : (fs_out / 2),
#     "fir_numtaps"    : 129,
#     "zero_phase"     : True,
#     "out_filename" :  out_filename
# }
#     if bSetUpDB:
#         etl = CapnoBaseETL(config)
#         h5file = etl.process_all()
#         print(f"Saved windows HDF5 to {h5file}")
#     df = load_as_df(out_path,out_filename)

#     display(df.head(10))
#     return df
# if __name__ == "__main__":
#     df_capnobase = main()

## MIMIC III

### MIMIC III General Subset

In [11]:
from pathlib import Path
def main():
    ver_num = 3
    num_subjects = 2100
    set_filenames = sorted(glob.glob(f"data/raw/mimic3_data/subject_sets/mimic3_data_{num_subjects}_*_v{ver_num}.h5"))
    num_sets = 5
    # root_path = os.path.join(f'data/raw/mimic3_data/{filename}')
    for file_paths in set_filenames[:num_sets]:
        root_path = file_paths
        filename = file_paths.rpartition('/')[-1]
        out_filename = f'{filename.rpartition('.')[0]}'
        out_folder =  f'mimic3_db_v{ver_num}_sets{FOLDER_NAME}/'
        out_path = Path('data/processed/length_full/') / out_folder

        if not out_path.exists():
            # os.mkdir(out_path)
            out_path.mkdir(parents=True,exist_ok=True)
        config = {
            "input_dir": root_path,
            "output_dir":  out_path,
            "fs_in": 125.00,
            "fs_out": 20.83,
            'fs_ekg': 125,
            'fs_bp':125,
            "out_filename": out_filename,
            "window_size_sec": WINDOW_LEN,
            "scale_type": SCALE_TYPE,
            "decimate_signal": DECIMATE_SIGNAL,
            "zero_phase": ZEROPHASE,
            "lowpass_cutoff":LOW_PASS_CUTOFF,
            "fir_numtaps":FIR_NUMTAPS,
            
        }
        
        bSetUpDB = False
        if bSetUpDB:
            etl = MimicETL(config)
            out_file = etl.process()
            print("Saved General MIMIC III subject windows set to: ", out_file)
    df = load_as_df(out_path,out_filename)
    display(df)
    # df
    return
if __name__ == "__main__":
    main()
    # first = df_mimic3_gen.iloc[np.random.randint(0, len(df_mimic3_gen))] 
#     chosen_subj = df_mimic3_gen['subject'].unique()[0]

#     surr_subject_df = df_mimic3_gen[df_mimic3_gen['subject'] == chosen_subj]

#     display(surr_subject_df.head())
#     initial_window = df_mimic3_gen.sample(1).iloc[0]

#     specs = [
#         {"key":"raw_ppg",  "fs_key":"raw_ppg_fs",  "subplot":1, "legend":"Raw PPG"},
#         {"key":"proc_ppg", "fs_key":"ppg_fs_out",  "subplot":1, "legend":"Proc PPG"},
#         {"key":"raw_ekg",  "fs_key":"ekg_fs_out",  "subplot":2,               },
#         {"key":"raw_abp",  "fs_key":"abp_fs_out",  "subplot":3,               },
#     ]

#     fig_fn = lambda window: make_window_figure(window, specs)
    
#     app = create_time_series_viewer(
#     surr_subject_df,
#     fig_fn,
#     specs,
#     index_label_fn=lambda idx, row: f"{row.subject}-{row.window_count}"
# )
#     print(f'Total # of Subjects: {len(df_mimic3_gen['subject'].unique())}')
#     print(f'Total # of Windows: {df_mimic3_gen.shape[0]}')
#     print("Launching Dash at http://127.0.0.1:8050 …")
#     app.run(debug=True)

Loading: data/processed/length_full/mimic3_db_v3_sets_processed/mimic3_data_2100_14_v3.h5


,subject,window_id,window_count,rec_id,label,raw_ppg_fs,ppg_fs_out,ekg_fs_out,abp_fs_out,raw_len,proc_len,duration_raw_s,duration_proc_s,notes,raw_ppg,proc_ppg,raw_ekg,raw_abp
0,p072790,02821994-4948-4b19-b45b-8d8d9ecfc229,1,p072790-2125-06-11-22-20,-1,125.0,20.83,125.0,125.0,1000,167,48.007681,8.017283,,"[0.43597263, 0.4848485, 0.53274685, 0.5786901,...","[-0.8398234, 1.7547693, 1.73114, 1.8730177, 1....","[0.38039216, 0.38039216, 0.3745098, 0.37058824...","[129.6611, 127.90099, 126.72759, 125.55419, 12..."
1,p072790,04465b4d-65ce-4aa6-8cfd-3591a17f99a1,2,p072790-2125-06-11-22-20,-1,125.0,20.83,125.0,125.0,1000,167,48.007681,8.017283,,"[0.69501466, 0.7038123, 0.7106549, 0.71652, 0....","[-0.16854554, 2.4001863, 1.7729387, 2.0070534,...","[0.35490197, 0.35490197, 0.3647059, 0.3647059,...","[120.27387, 119.68717, 118.9049, 118.3182, 117..."
2,p072790,0674b587-fb5a-4317-a461-e46479f1f280,3,p072790-2125-06-11-22-20,-1,125.0,20.83,125.0,125.0,1000,167,48.007681,8.017283,,"[0.5522972, 0.542522, 0.53274685, 0.52394915, ...","[-0.6034916, 1.2102215, 0.36070943, 0.38683462...","[0.7411765, 0.7352941, 0.7509804, 0.7411765, 0...","[97.588066, 97.19694, 96.61023, 96.2191, 95.82..."
3,p072790,06a742aa-d415-4b7f-99d0-79d4853d5857,4,p072790-2125-06-11-22-20,-1,125.0,20.83,125.0,125.0,1000,167,48.007681,8.017283,,"[0.70185727, 0.7096774, 0.7155425, 0.72043014,...","[-0.42975104, 2.1023395, 1.4841112, 1.6407678,...","[0.3745098, 0.38431373, 0.38431373, 0.39411765...","[116.5581, 116.16696, 115.58026, 114.99355, 11..."
4,p072790,0706b5b5-6d0e-40bd-ac6e-d316775f2f9f,5,p072790-2125-06-11-22-20,-1,125.0,20.83,125.0,125.0,1000,167,48.007681,8.017283,,"[0.5796676, 0.5786901, 0.5777126, 0.57575756, ...","[-0.9109033, 1.7809527, 0.6658186, 0.5960577, ...","[0.40392157, 0.4, 0.40392157, 0.4, 0.40392157,...","[113.82015, 113.23345, 112.842316, 112.45118, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9342,p074763,f222ab97-9d98-4980-8547-3fe6704bb770,183,p074763-2111-06-14-07-57,-1,125.0,20.83,125.0,125.0,1000,167,48.007681,8.017283,,"[0.3137255, 0.3137255, 0.3137255, 0.3137255, 0...","[-1.9371125, -0.75374657, -1.0192915, -0.93140...","[0.3828125, 0.375, 0.375, 0.375, 0.375, 0.375,...","[142.1875, 140.625, 140.625, 141.40625, 141.40..."
9343,p074763,f67615d4-5e46-4bb3-9fcd-ec3507cca1e9,184,p074763-2111-06-14-07-57,-1,125.0,20.83,125.0,125.0,1000,167,48.007681,8.017283,,"[0.35686275, 0.3529412, 0.34509805, 0.34117648...","[-1.5546929, -0.54926807, -0.9807393, -0.90036...","[0.421875, 0.4375, 0.453125, 0.4765625, 0.5, 0...","[53.125, 52.34375, 51.5625, 50.78125, 50.0, 50..."
9344,p074763,f6ddcb45-7c6e-4625-a106-fada9db8ca1d,185,p074763-2111-06-14-07-57,-1,125.0,20.83,125.0,125.0,1000,167,48.007681,8.017283,,"[0.35686275, 0.35686275, 0.3529412, 0.3529412,...","[-2.056883, -0.6208893, -1.0844312, -0.8971500...","[0.4140625, 0.40625, 0.40625, 0.40625, 0.40625...","[151.5625, 150.0, 146.875, 144.53125, 141.4062..."
9345,p074763,f86d87cf-3c99-4d8c-9c35-8c9e02aa131e,186,p074763-2111-06-14-07-57,-1,125.0,20.83,125.0,125.0,1000,167,48.007681,8.017283,,"[0.2784314, 0.27450982, 0.27450982, 0.27058825...","[-1.6577567, -0.8679273, -1.0683255, -0.940714...","[0.4609375, 0.4296875, 0.4140625, 0.40625, 0.4...","[173.4375, 164.0625, 154.6875, 148.4375, 145.3..."


In [ ]:
# ver_num = 1
# out_filename = f'mimic3_db_v{ver_num}_test'
# out_path = os.path.join(f'data/processed/length_full/mimic3_db_v{ver_num}_test')    
# h5_path = f"{out_path}/{out_filename}.h5"
# print(f'Loading: {h5_path}')
# hf = h5py.File(h5_path, "r")
# subj_id = list(hf.keys())[0]
# window_id = list(hf[subj_id].keys())[0]
# print(dict(hf[subj_id][window_id].attrs))


## MIMIC IV

### MIMIC IV General Data

In [ ]:
np.random.seed(42)
ver_num = 3
set_filenames = sorted(glob.glob(f"data/raw/mimic4_data/subject_sets/mimic4_data_53_*_v{ver_num}.h5"))
def main():
    # root_path = os.path.join(f'data/raw/mimic3_data/{filename}')
    for file_paths in set_filenames:
        root_path = file_paths
        filename = file_paths.rpartition('/')[-1]
        out_filename = f'{filename.rpartition('.')[0]}'
        out_folder =  f'mimic4_db_v{ver_num}_sets{FOLDER_NAME}/'
        out_path = Path('data/processed/length_full/') / out_folder
    

        if not out_path.exists():
            out_path.mkdir(parents=True,exist_ok=True)
        fs_in = 62.5
        fs_out = 20.83
        config = {
            "input_dir": root_path,
            "output_dir":  out_path,
            "fs_in": fs_in,
            "fs_out": fs_out,
            'fs_ekg': 62.5, 
            'fs_bp':fs_in,
            "window_size_sec": WINDOW_LEN,
            "out_filename": out_filename,
            "scale_type": SCALE_TYPE,
            "decimate_signal": DECIMATE_SIGNAL,
            "zero_phase": ZEROPHASE,
            "lowpass_cutoff":LOW_PASS_CUTOFF,
            "fir_numtaps":FIR_NUMTAPS,
            
        }
        bSetUpDB = True
        if bSetUpDB:
            etl = MimicETL(config)
            out_file = etl.process()
            print(f"Saved MIMIC IV  windows to", out_file)
        df = load_as_df(out_path,out_filename)
        display(df)
        return df

if __name__ == "__main__":

    df_mimic4_gen = main()
#     first = df_mimic4_gen.iloc[np.random.randint(0, len(df_mimic4_gen))] 
#     initial_window = df_mimic4_gen.sample(1).iloc[0]
#     specs = [
#         {"key":"raw_ppg",  "fs_key":"raw_ppg_fs",  "subplot":1, "legend":"Raw PPG"},
#         {"key":"proc_ppg", "fs_key":"ppg_fs_out",  "subplot":1, "legend":"Proc PPG"},
#         {"key":"raw_ekg",  "fs_key":"ekg_fs_out",  "subplot":2,               },
#         {"key":"raw_abp",  "fs_key":"abp_fs_out",  "subplot":3,               },
#     ]

#     # 3) bind specs into a single-arg figure fn
#     fig_fn = lambda window: make_window_figure(window, specs)
    
#     app = create_time_series_viewer(
#     df_mimic4_gen,
#     fig_fn,
#     specs,
#     index_label_fn=lambda idx, row: f"{row.subject}-{row.window_count}"
# )
#     print(f'Total # of Subjects: {len(df_mimic4_gen['subject'].unique())}')
#     print(f'Total # of Windows: {df_mimic4_gen.shape[0]}')
#     print("Launching Dash at http://127.0.0.1:8050 …")
    
    # app.run(debug=True)
        

Saved MIMIC IV  windows to /Users/shayanriyaz/Documents/Projects/heart_rhythm_analysis/data/processed/length_full/mimic4_db_v3_sets_processed/mimic4_data_53_1_v3.h5
Loading: data/processed/length_full/mimic4_db_v3_sets_processed/mimic4_data_53_1_v3.h5


,subject,window_id,window_count,rec_id,label,raw_ppg_fs,ppg_fs_out,ekg_fs_out,abp_fs_out,raw_len,proc_len,duration_raw_s,duration_proc_s,notes,raw_ppg,proc_ppg,raw_ekg,raw_abp
0,p10020306,00698a6d-aa9c-4685-b3a0-c817d77b8de0,1,83404654,-1,62.4725,20.83,62.5,62.5,499,167,23.955833,8.017283,,"[0.4873047, 0.4790039, 0.47070312, 0.46240234,...","[-2.5711114, 0.92433476, -0.70122284, -0.57906...","[-0.015, -0.02, -0.015, -0.015, -0.015, -0.02,...","[65.4375, 64.6875, 63.9375, 63.25, 62.625, 62...."
1,p10020306,0245800f-8ccd-4bbb-884d-6331782e00d8,2,83404654,-1,62.4725,20.83,62.5,62.5,499,167,23.955833,8.017283,,"[0.4177246, 0.43188477, 0.4428711, 0.45336914,...","[-3.0067742, -0.018481176, -0.59240705, -0.385...","[-0.065, -0.065, -0.06, -0.055, -0.08, -0.07, ...","[58.25, 72.5, 93.6875, 110.625, 121.0625, 126...."
2,p10020306,02eec7a3-7c74-41cf-b908-bf7e52c8ea56,3,83404654,-1,62.4725,20.83,62.5,62.5,499,167,23.955833,8.017283,,"[0.5083008, 0.5083008, 0.5, 0.5, 0.5, 0.491455...","[-2.029374, 1.5777515, 0.23545568, 0.49812907,...","[-0.065, -0.05, -0.045, -0.035, -0.03, -0.015,...","[54.75, 56.9375, 64.125, 75.5, 86.4375, 93.937..."
3,p10020306,06cc3808-16b0-4ea3-b112-6c00375e9d75,4,83404654,-1,62.4725,20.83,62.5,62.5,499,167,23.955833,8.017283,,"[0.36767578, 0.3605957, 0.3569336, 0.34985352,...","[-2.048538, -0.7935949, -1.114626, -0.26635954...","[-0.005, -0.025, -0.035, -0.03, -0.05, -0.04, ...","[91.3125, 90.75, 88.875, 85.75, 81.25, 76.1875..."
4,p10020306,07500c80-f80f-48d5-a119-c910c55a8702,5,83404654,-1,62.4725,20.83,62.5,62.5,499,167,23.955833,8.017283,,"[0.56225586, 0.5539551, 0.5456543, 0.5373535, ...","[-1.492038, 0.9779711, -0.09438914, 0.1646142,...","[-0.055, -0.06, -0.045, -0.02, -0.015, -0.01, ...","[62.0, 67.9375, 75.9375, 83.3125, 87.75, 89.12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9341,p18824975,f877627d-c744-439c-907e-67e1298c0ffb,182,86288257,-1,62.4725,20.83,62.5,62.5,499,167,23.955833,8.017283,,"[0.5061035, 0.5192871, 0.52783203, 0.5324707, ...","[-0.5223602, 0.52019614, 0.29092622, 0.6248494...","[0.05, 0.05, -0.02, -0.015, 0.025, 0.0, 0.06, ...","[94.8125, 98.0, 92.3125, 92.4375, 92.125, 86.3..."
9342,p18824975,fab77879-9edc-4f01-8388-14c512e96ebb,183,86288257,-1,62.4725,20.83,62.5,62.5,499,167,23.955833,8.017283,,"[0.4934082, 0.52612305, 0.55249023, 0.5695801,...","[-0.4666306, 0.4801187, 0.25915426, 0.16022092...","[0.02, -0.005, -0.01, -0.015, -0.015, 0.01, 0....","[76.9375, 71.125, 65.5625, 61.25, 58.125, 55.6..."
9343,p18824975,fb693fb6-e109-45ac-913e-ab7034f5d071,184,86288257,-1,62.4725,20.83,62.5,62.5,499,167,23.955833,8.017283,,"[0.5358887, 0.5317383, 0.5263672, 0.5209961, 0...","[-1.2195526, -0.19531633, -0.57763547, -0.4413...","[-0.055, -0.105, -0.115, -0.1, -0.085, -0.08, ...","[53.8125, 54.4375, 54.8125, 55.3125, 55.5, 55...."
9344,p18824975,fccf7c6a-30ff-49aa-995a-84a0a3f17259,185,86288257,-1,62.4725,20.83,62.5,62.5,499,167,23.955833,8.017283,,"[0.4741211, 0.46899414, 0.46899414, 0.4638672,...","[-1.4947635, -0.120434545, -0.6134063, -0.5551...","[-0.04, -0.055, -0.055, -0.06, -0.07, -0.06, -...","[81.5625, 78.375, 75.4375, 72.75, 69.875, 66.6..."


In [ ]:
# subject_agg = {
#     # sampling & metadata: just take the first (they're constant per subject)
#     "raw_ppg_fs": "first",
#     "ppg_fs_out": "first",
#     "ekg_fs_out": "first",
#     "abp_fs_out": "first",
#     # lengths & durations: sum across windows
#     "raw_len":       "sum",
#     "proc_len":      "sum",
#     "duration_raw_s":  "sum",
#     "duration_proc_s": "sum",
#     # signals: concatenate all windows end-to-end
#     "raw_ppg":  lambda s: np.concatenate(s.values),
#     "proc_ppg": lambda s: np.concatenate(s.values),
#     "raw_ekg":  lambda s: np.concatenate(s.values),
#     "raw_abp":  lambda s: np.concatenate(s.values),
#     # if you want to keep track of window_ids or rec_ids:
#     "window_id": lambda s: list(s.values),
#     "rec_id":    lambda s: list(s.values),
#     "label":     lambda s: list(s.values),
# }

# # ── 2) group by subject ─
# df_by_subject = (
#     df_mimic4_gen
#     .groupby("subject", as_index=False)
#     .agg(subject_agg)
# )

# # ── 3) now df_by_subject has one row per subject, with each signal a long array
# print(df_by_subject.shape)
# print(df_by_subject.raw_ppg.iloc[0].shape)  # e.g. (sum of all its windows,) 

# # bind specs & figure fn exactly as before
# fig_fn = lambda row: make_window_figure(row, specs)

# # subject‐level dashboard (labels are just the subject IDs)
# app = create_time_series_viewer(
#     df_by_subject,
#     fig_fn,
#     specs,
#     index_label_fn=lambda idx, row: str(row.subject)
# )
# print("Launching Dash at http://127.0.0.1:8080 …")
# app.run(debug=True,port=8080)

# Create Train/Test Files

In [22]:
from src.lib.create_training_pipelines.prepare_dataset import *
import os, h5py

ver_num = 3
ROOT_PATH = 'data/processed/length_full'

mimic3_files = sorted(glob.glob(f'{ROOT_PATH}/mimic3_db_v{ver_num}_sets{FOLDER_NAME}/*'))
mimic4_files = sorted(glob.glob(f'{ROOT_PATH}/mimic4_db_v{ver_num}_sets{FOLDER_NAME}/*'))

file_paths = mimic3_files + mimic4_files
# file_paths = [f"{ROOT_PATH}/mimic3_db_v{ver_num}/mimic3_db_v{ver_num}.h5", f"{ROOT_PATH}/mimic4_db_v{ver_num}/mimic4_db_v{ver_num}.h5"]
out_path =Path('data/development_dataset/length_full')
out_path.mkdir(parents=True,exist_ok=True)
train_path, test_path = my_split_and_save(file_paths, train_ratio=0.8, output_dir=out_path)

def count_windows(h5_path):
    with h5py.File(h5_path, 'r') as f:
        subjects = list(f.keys())
        n_subjects = len(subjects)
        n_windows  = sum(len(f[subj].keys()) for subj in subjects)
    return n_subjects, n_windows



In [23]:
# Replace these with your real paths:
paths = {
    "train":   "data/development_dataset/length_full/train_dataset.h5",
    "test":    "data/development_dataset/length_full/test_dataset.h5",
}

for name, p in paths.items():
    if os.path.exists(p):
        subs, wins = count_windows(p)
        print(f"{name:>6} → {subs} subjects, {wins} windows")
    else:
        print(f"{name:>6} → file not found: {p}")

 train → 2089 subjects, 311269 windows
  test → 2089 subjects, 77818 windows
